In [ ]:
import requests
import re
import urllib 
from bs4 import BeautifulSoup
import string
import pandas as pd
import numpy as np
import datetime
import _datetime
import time

In [ ]:

headers = requests.utils.default_headers()
headers['User-Agent'] = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"


In [ ]:

url='https://avtolavka.net'
r = requests.get(url, headers=headers)

html_content = r.text
soup = BeautifulSoup(html_content,'html.parser')
tuy=[]
for y in soup.findAll('a',class_='headCatalog-item-link'):
    tuy.append(y.get('href'))
tuy=np.unique(tuy)

In [ ]:
price=[]
kolvo=[]
time=[]
name_all=[]
article_all=[]
brand_all=[]
url_all=[]
musor=[]

In [ ]:
len(tuy)

In [ ]:
for h in range(len(tuy)):
    print(h,'cat')
    url='https://avtolavka.net'+tuy[h]
    r = requests.get(url, headers=headers)

    html_content = r.text
    soup0 = BeautifulSoup(html_content,'html.parser')
    if h==7:
        ko=166
    else:
        ko=0
    if soup0.findAll('li',class_='last')!=[]:
        a=soup0.findAll('li',class_='last')[0].find('a').get('href')
        ui=a.find('start=')
        last_page=int(int(a[ui+6:])/20)+1
        print('last page - ',last_page)
        for r in range(ko,last_page):
            print(r)
            url0=url+'?start='+str(r*20)
            r0 = requests.get(url0, headers=headers)

            html_content0 = r0.text
            soup = BeautifulSoup(html_content0,'html.parser')

            all_prod=soup.findAll('li',class_='item')
            name=''
            article=''
            brand=''
            for u in range(len(all_prod)):
                positio=all_prod[u]
                if positio.findAll('a',target="_blank")!=[]:
                    name=positio.findAll('a',target="_blank")[0].text.replace('\t','').replace('\n','')
                    lin=positio.findAll('a',target="_blank")[0].get('href')
                else:
                    musor.append(url0)
                if positio.findAll('a',rel="nofollow")!=[]:
                    article=positio.findAll('a',rel="nofollow")[0].text
                if positio.findAll('a',class_='brandInfoLink')!=[]:    
                    brand=positio.findAll('a',class_='brandInfoLink')[0].text.replace('\n                                                    ','').replace('\t','').replace('\n','')



                if positio.findAll('div',class_="price_block_in")!=[]:
                    price_osn=positio.findAll('div',class_="price_block_in")[0].findAll('td')
                    time.append(price_osn[0].text)
                    kolvo.append(price_osn[1].text)
                    price.append(price_osn[2].text.replace('\n','').replace('\t',''))
                    name_all.append(name)
                    article_all.append(article)
                    brand_all.append(brand)
                    if positio.findAll('div',class_='hidden-price-wrap hidden')!=[]:
                        len_price=positio.findAll('div',class_='hidden-price-wrap hidden')[0].findAll('td')
                        t=int(len(len_price)/3)


                        for j in range(t):
                            time.append(len_price[j*3].text)
                            kolvo.append(len_price[j*3+1].text)
                            price.append(len_price[j*3+2].text.replace('\n','').replace('\t',''))
                            name_all.append(name)
                            article_all.append(article)
                            brand_all.append(brand)
                            url_all.append(lin)

                else:
                    time.append('')
                    kolvo.append('')
                    price.append('')
                    name_all.append(name)
                    article_all.append(article)
                    brand_all.append(brand)
                    url_all.append(url0)




In [ ]:
table = pd.DataFrame(list(zip(name_all,article_all,brand_all,time,kolvo,price,url_all))) 

table.to_excel('avtolavka_250621.xlsx')